In [ ]:
!pip install faiss-cpu

#Imports

In [ ]:
import numpy as np
from scipy import spatial
import faiss
from time import time

from collections import defaultdict
from sklearn.cluster import KMeans

## Helper Function

In [ ]:
def semi_optimized_exhaustive_search(
        index_vectors: np.ndarray,
        query_vectors: np.ndarray,
        k: int,
):
    """
    This function performs an optimized exhaustive search.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        query_vectors: An array of shape (n_queries, dim) containing the query vectors.
        dim: The dimensionality of the vectors.
    Returns:
        An array of shape (n_queries, k) containing the indices of the k nearest neighbors for each query vector.
    """
    ann_lists = []
    for query_vec in query_vectors:
        distances = np.linalg.norm(index_vectors - query_vec, axis=1)
        ann_lists.append(list(np.argsort(distances)[:k]))
    return np.array(ann_lists)

In [ ]:
def build_faiss_flatl2_index(
        index_vectors: np.ndarray,
        dim: int,
):
    """
    This function builds a Faiss flat L2 index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors.
    Returns:
        A Faiss flat L2 index.
    """
    index = faiss.IndexFlatL2(dim)
    index.add(index_vectors)
    return index

In [ ]:
def faiss_search(
        query_vectors: np.ndarray,
        index: faiss.Index,
        k: int,
):
    """
    This function uses a Faiss index to search for the k-nearest neighbors of query_vectors.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors.
        index: A Faiss index.
        k: The number of nearest neighbors to retrieve.
    Returns:
        An array of shape (, ) containing the indices of the k-nearest neighbors for each query vector.
    """
    distances, indices = index.search(query_vectors, k)
    return indices

In [ ]:
def build_faiss_lsh_index(
        index_vectors: np.ndarray,
        dim: int,
        nbits: int,
):
    """
    This function builds a Faiss LSH index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors.
        nbits: The number of bits to use in the hash.
    Returns:
        A Faiss LSH index.
    """
    index = faiss.IndexLSH(dim, nbits)
    index.add(index_vectors)
    return index

In [ ]:
def compute_recall_at_k(
        nn_gt: np.ndarray,
        ann: np.ndarray,
        k: int,
):
    """
    This function computes the recall@k.
    Args:
        nn_gt: The ground truth nearest neighbors.
        ann: The approximate nearest neighbors.
        k: The number of nearest neighbors to consider.
    Returns:
        The recall@k.
    """
    return round(sum([len(set(ann[i]) & set(nn_gt[i])) / k for i in range(len(ann))])/len(ann), 3)

# 2.1 -- LSH vs Naive Exhaustive Search (Regular Index Vectors)
### You just have to run the following cells and add the following results to the report:
* running time of the ground truth computation with semi_optimized_exhaustive_search (wall time)
* running time of creating faiss_lsh_index (wall time)
* running time of faiss_search over query_vectors with faiss_lsh_index (wall time)
* recall@10 for faiss_lsh_ann

In [ ]:
query_vectors = np.load('/content/query_vectors.npy')
index_vectors = np.load('/content/index_vectors.npy')
k=10
dim = index_vectors.shape[1]

In [ ]:
%%time
print("part 2.1.1")
gt_nn = semi_optimized_exhaustive_search(index_vectors, query_vectors, k)

part 2.1.1
CPU times: user 11.6 s, sys: 10.8 s, total: 22.4 s
Wall time: 35.2 s


In [ ]:
%%time
print("part 2.1.2")
faiss_lsh_index = build_faiss_lsh_index(index_vectors, dim, nbits=2000)

part 2.1.2
CPU times: user 1.81 s, sys: 241 ms, total: 2.05 s
Wall time: 3.28 s


In [ ]:
%%time
print("part 2.1.3")
faiss_lsh_ann = faiss_search(query_vectors, faiss_lsh_index, k)

part 2.1.3
CPU times: user 1.27 s, sys: 709 µs, total: 1.27 s
Wall time: 1.29 s


In [ ]:
print("part 2.1.4")
print(f"recall@10 for faiss_lsh_index: {compute_recall_at_k(gt_nn, faiss_lsh_ann, k)}")

part 2.1.4
recall@10 for faiss_lsh_index: 0.138


# 2.2 -- Custom Indexing Algorithm
Build an indexing algorithm that satisfies the following requirements:
* The indexing algorithm should be able to handle vectors of different dimensions
* The running time of the indexing should be less than half of the running time of semi_optimized_exhaustive_search), reported in Section 2.1.
* The running time of searching over the index should be less than a third (1/3) of the time of the semi_optimized_exhaustive_search function, reported in Section 2.1.
* The performance (in terms of recall@10) of the indexing algorithm should be at least 0.8.

The last three bullets should also appear in the report.
You are allowed to add as many helper functions as you need. You cannot use faiss of scipy libraries for this task. Numpy is allowed.

You can also test your algorithm with the additional two query-index sets by replacing the calls made few cells ago to:

    query_vectors = np.load('data/query_vectors2.npy')
    index_vectors = np.load('data/index_vectors2.npy')
or:

    query_vectors = np.load('data/query_vectors3.npy')
    index_vectors = np.load('data/index_vectors3.npy')
    
the aforementioned requirements should also be satisfied over these two query-index sets. No need to insert the results over these two to the report.

#Help Function For Indexing Function

In [ ]:
# Performs k-means clustering on vectors
def k_M(n_clusters, vectors):
    """
    This function create the custom index.
    Args:
        n_clusters: The number of clusters to form.
        vectors: An array of shape (n_vectors, dim) containing the index vectors.
    """
    #res: a dictionary that will store the indices of the input vectors, grouped by the cluster they belong to
    res={}

    kmeans = KMeans(n_clusters=n_clusters, max_iter=100)
    kmeans.fit(vectors)

    #getting the cluster labels assigned to each data point after fitting the model (ranging from 0 to k-1)
    labs = kmeans.labels_
    for i, lab in enumerate(labs):
        #checking if we need to create a new key to a label
        if lab not in res:
            res[lab] = []
        #the index of the data point is appended to the list corresponding to its cluster label
        res[lab].append(i)

    return res, kmeans

#Help Function for Custom Search

In [ ]:
def closest_k(q_indexes, i_indexes, k, query_vectors, index_vectors):
    """
    This function finds the closest k vectors from a set of index vectors for each vector in a set of query vectors.
    Args:
        q_indexes: The indexes of the query vectors.
        i_indexes: The indexes of the index vectors.
        k: The number of nearest neighbors to retrieve.
        query_vectors: An array of shape (n_queries, dim) containing the query vectors.
        index_vectors: An array of shape (n_vectors, dim) containing the index vectors.
    """
    #selects a subset of vectors from query_vectors and index_vectors
    selected_i = index_vectors[i_indexes]
    selected_q = query_vectors[q_indexes]

    #pairwise Euclidean distances between vectors in selected_q and selected_i
    dis = np.linalg.norm(selected_q[:, np.newaxis] - selected_i, axis=2)

    #partially sorts the distances along the specified axis in order to get the k smallest elements
    smallest_k = np.argpartition(dis, k, axis=1)[:, :k]

    #res: a list of tuples. tuple contains: list of indices of k nearest index vectors to each query vector.
    #The index of the query vector itself.
    res = []
    for i, q_idx in enumerate(q_indexes):
        #finding the K closest indeices
        closest = [i_indexes[idx] for idx in smallest_k[i]]

        res.append((q_idx, closest))

    return res

In [ ]:
def custom_indexing_algorithm(index_vectors, dim, n_clusters):
    """
    This function create the custom index.
    Args:
        index_vectors: An array of shape (n_vectors, dim) containing the index vectors.
        dim: The custom index.
        n_clusters: The required number of clusters.
    """
    dict_i, trained_km = k_M(n_clusters, index_vectors)

    return dict_i, trained_km


def custom_index_search(query_vectors, k, dict_i, index_vectors, trained_km):
    """
    This function searches over the custom index.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors.
        k: The number of nearest neighbors to retrieve.
        dict_i: The code to index dictionary.
        index_vectors: An array of shape (n_vectors, dim) containing the index vectors.
        trained_km: The trained k-means model.
    """
    res=[]

    #indeices_to_q: a dictionary that will store the indices of the input vectors, grouped by the cluster they belong to
    indeices_to_q={}

    #initializes with an empty list for each cluster label
    for clus_index in range(len(dict_i)):
      indeices_to_q[clus_index]=[]

    #predicting cluster label using K-means model and adding index to the corresponding list in indeices_to_q
    for i, q_vec in enumerate(query_vectors):
        q_lab = trained_km.predict(q_vec.reshape(1, -1))
        #the index of the data point is appended to the list corresponding to its cluster label
        indeices_to_q[q_lab[0]].append(i)

    for v_to_q in indeices_to_q.keys():
      #retrieves the corresponding index_vectors and query_vectors
      index_idxs = dict_i[v_to_q]
      query_idxs = indeices_to_q[v_to_q]

      #finding the closest k index_vectors to each query_vector
      closest = closest_k(query_idxs, index_idxs, k, query_vectors, index_vectors)
      res.extend(closest)

    #sorting based on query_vector's index
    res.sort(key = lambda x: x[0])
    #list of lists - each inner list contains the indices of the k nearest index vectors for each query_vector
    res = [x[1] for x in res]

    #casting list of lists to nparray
    return np.array(res)

In [ ]:
%%time
print("part 2.2.2")
clus_num=10
custom_index, trained_kmeans = custom_indexing_algorithm(index_vectors, dim, clus_num)

part 2.2.2


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


CPU times: user 1.57 s, sys: 533 ms, total: 2.1 s
Wall time: 2.11 s


In [ ]:
%%time
print("part 2.2.3")
custom_index_ann = custom_index_search(query_vectors , 10, custom_index, index_vectors, trained_kmeans)

part 2.2.3
CPU times: user 1.34 s, sys: 985 ms, total: 2.33 s
Wall time: 3.24 s


In [ ]:
print("part 2.2.4")
print(f"recall@10 for custom_index_search: {compute_recall_at_k(gt_nn, custom_index_ann, k)}")

part 2.2.4
recall@10 for custom_index_search: 1.0


#Testing on query_vectors2/3 and index_vectors2/3

In [ ]:
"""
query_vectors2 = np.load('/content/query_vectors2.npy')
index_vectors2 = np.load('/content/index_vectors2.npy')
%%time
custom_index2, trained_kmeans2 = custom_indexing_algorithm(index_vectors2, 100, clus_num)
%%time
custom_index_ann = custom_index_search(query_vectors2 , 10, custom_index2, index_vectors2, trained_kmeans2)
%%time
gt_nn2 = semi_optimized_exhaustive_search(index_vectors2, query_vectors2, k)
print(f"recall@10 for custom_index_search: {compute_recall_at_k(gt_nn2, custom_index_ann, k)}")
"""

'\nquery_vectors2 = np.load(\'/content/query_vectors2.npy\')\nindex_vectors2 = np.load(\'/content/index_vectors2.npy\')\n%%time\ncustom_index2, trained_kmeans2 = custom_indexing_algorithm(index_vectors2, 100, clus_num)\n%%time\ncustom_index_ann = custom_index_search(query_vectors2 , 10, custom_index2, index_vectors2, trained_kmeans2)\n%%time\ngt_nn2 = semi_optimized_exhaustive_search(index_vectors2, query_vectors2, k)\nprint(f"recall@10 for custom_index_search: {compute_recall_at_k(gt_nn2, custom_index_ann, k)}")\n'

In [ ]:
"""
query_vectors3 = np.load('/content/query_vectors3.npy')
index_vectors3 = np.load('/content/index_vectors3.npy')
%%time
custom_index3, trained_kmeans3 = custom_indexing_algorithm(index_vectors3, 200, clus_num)
%%time
custom_index_ann = custom_index_search(query_vectors3 , 10, custom_index3, index_vectors3, trained_kmeans3)
%%time
gt_nn3 = semi_optimized_exhaustive_search(index_vectors3, query_vectors3, k)
print(f"recall@10 for custom_index_search: {compute_recall_at_k(gt_nn3, custom_index_ann, k)}")
"""

'\nquery_vectors3 = np.load(\'/content/query_vectors3.npy\')\nindex_vectors3 = np.load(\'/content/index_vectors3.npy\')\n%%time\ncustom_index3, trained_kmeans3 = custom_indexing_algorithm(index_vectors3, 200, clus_num)\n%%time\ncustom_index_ann = custom_index_search(query_vectors3 , 10, custom_index3, index_vectors3, trained_kmeans3)\n%%time\ngt_nn3 = semi_optimized_exhaustive_search(index_vectors3, query_vectors3, k)\nprint(f"recall@10 for custom_index_search: {compute_recall_at_k(gt_nn3, custom_index_ann, k)}")\n'